In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
df = pd.read_csv('/content/anime.csv')
df.fillna('', inplace=True)
print(df.head())


   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes rating  \
0               Drama, Romance, School, Supernatural  Movie        1   9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64   9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51   9.25   
3                                   Sci-Fi, Thriller     TV       24   9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51   9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


<ipython-input-44-9f6a837e57d4>:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [45]:
df['genre'].fillna('Unknown', inplace=True)
df['type'].fillna('Unknown', inplace=True)

df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['rating'].fillna(df['rating'].mean(), inplace=True)

df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
df['episodes'].fillna(0, inplace=True)

df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  float64
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 672.5+ KB


(None,
    anime_id                              name  \
 0     32281                    Kimi no Na wa.   
 1      5114  Fullmetal Alchemist: Brotherhood   
 2     28977                          Gintama°   
 3      9253                       Steins;Gate   
 4      9969                     Gintama&#039;   
 
                                                genre   type  episodes  rating  \
 0               Drama, Romance, School, Supernatural  Movie       1.0    9.37   
 1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV      64.0    9.26   
 2  Action, Comedy, Historical, Parody, Samurai, S...     TV      51.0    9.25   
 3                                   Sci-Fi, Thriller     TV      24.0    9.17   
 4  Action, Comedy, Historical, Parody, Samurai, S...     TV      51.0    9.16   
 
    members  
 0   200630  
 1   793665  
 2   114262  
 3   673572  
 4   151266  )

In [46]:

tfidf_genre = TfidfVectorizer(stop_words='english')
tfidf_type = TfidfVectorizer(stop_words='english')

genre_matrix = tfidf_genre.fit_transform(df['genre'])
type_matrix = tfidf_type.fit_transform(df['type'])

genre_df = pd.DataFrame(genre_matrix.toarray(), index=df.index, columns=tfidf_genre.get_feature_names_out())
type_df = pd.DataFrame(type_matrix.toarray(), index=df.index, columns=tfidf_type.get_feature_names_out())

numerical_features = df[['episodes', 'rating', 'members']]
combined_features = pd.concat([numerical_features, genre_df, type_df], axis=1)

scaler = StandardScaler()
normalized_features = scaler.fit_transform(combined_features)

combined_features.shape, normalized_features.shape


((12294, 55), (12294, 55))

In [47]:
train_df, test_df, train_features, test_features = train_test_split(
    df, normalized_features, test_size=0.2, random_state=42
)
cosine_sim = cosine_similarity(train_features)
def get_recommendations(anime_id, train_df, cosine_sim, top_n=10):
    if anime_id not in train_df['anime_id'].values:
        return f"No recommendations available for anime_id: {anime_id}"
    idx = train_df.index[train_df['anime_id'] == anime_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_indices = [i[0] for i in sim_scores[1:top_n+1]]
    return train_df.iloc[sim_indices][['anime_id', 'name', 'genre', 'rating']]